# MovingPandas Minimum Viable Example

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/movingpandas/movingpandas-examples/main?filepath=1-tutorials/99-mini-example.ipynb)

MovingPandas provides a trajectory datatype based on GeoPandas.
The project home is at https://github.com/movingpandas/movingpandas

The documentation is available at https://movingpandas.readthedocs.io/en/main/

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from hvplot import pandas
from datetime import datetime, timedelta
import folium

## Loading trajectory data from a GeoPackage

The MovingPandas repository contains a demo GeoPackage file that can be loaded as follows:

In [ ]:
gdf = gpd.read_file("../data/geolife_small.gpkg")
gdf.plot(figsize=(9, 5))

In [ ]:
gdf.hvplot(geo=True).opts(active_tools=["pan", "wheel_zoom"])

In [ ]:
gdf.explore()

After reading the trajectory point data from file, we want to construct the trajectories.

## Creating a TrajectoryCollection

In [ ]:
tc = mpd.TrajectoryCollection(gdf, "trajectory_id", t="t")
tc

In [ ]:
tc.plot(column="trajectory_id", legend=True, figsize=(9, 5))

In [ ]:
tc.explore(column="trajectory_id", cmap="plasma", style_kwds={"weight": 4})

## Exploring movement speed

In [ ]:
tc.plot(
    column="speed", linewidth=5, capstyle="round", legend=True, vmax=20, figsize=(9, 5)
)

## Detecting stops

In [ ]:
detector = mpd.TrajectoryStopDetector(tc)
stop_points = detector.get_stop_points(
    min_duration=timedelta(seconds=120), max_diameter=100
)

In [ ]:
ax = tc.plot(figsize=(9, 5))
stop_points.plot(ax=ax, color="red", markersize=100)

In [ ]:
(
    tc.hvplot(line_width=7, tiles=None, frame_width=400, frame_height=400)
    * stop_points.hvplot(geo=True, color="black", size=100)
)

In [ ]:
m = tc.explore(
    column="trajectory_id",
    cmap="autumn",
    style_kwds={"weight": 4},
    name="Trajectories",
)

stop_points.explore(
    m=m,
    marker_kwds={"radius": 4},
    name="Stop points",
)

folium.TileLayer("CartoDB dark_matter").add_to(m)
folium.LayerControl().add_to(m)

m